У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls drive/MyDrive/ML

application_data.csv.zip	customer_segmentation_train.csv  sample_submission.csv
application_data_processed.zip	medical-charges.csv		 test.csv
cars.csv			previous_application.csv.zip	 train.csv
columns_description.csv		regression_data.csv


In [4]:
data =pd.read_csv("drive/MyDrive/ML/customer_segmentation_train.csv")

In [5]:
data.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [6]:
data.shape

(8068, 11)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [8]:
print(data.isnull().sum())

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64


In [9]:
num_features = data.select_dtypes(include=['float64', 'int64']).columns
cat_features = data.select_dtypes(include=['object']).columns

In [10]:
cat_features

Index(['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score',
       'Var_1', 'Segmentation'],
      dtype='object')

In [11]:
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

In [12]:
data[num_features] = num_imputer.fit_transform(data[num_features])
data[cat_features] = cat_imputer.fit_transform(data[cat_features])

In [13]:
print(data.isnull().sum())

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64


In [14]:
# Кодування категоріальних змінних
categorical_columns = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
encoder = OneHotEncoder(sparse_output=False)
encoded_array = encoder.fit_transform(data[categorical_columns])

encoded_data = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(categorical_columns), index=data.index)

data = pd.concat([data.drop(categorical_columns, axis=1), encoded_data], axis=1)

print(data.head())


         ID   Age  Work_Experience  Family_Size Segmentation  Gender_Female  \
0  462809.0  22.0         1.000000          4.0            D            0.0   
1  462643.0  38.0         2.641663          3.0            A            1.0   
2  466315.0  67.0         1.000000          1.0            B            1.0   
3  461735.0  67.0         0.000000          2.0            B            0.0   
4  462669.0  40.0         2.641663          6.0            A            1.0   

   Gender_Male  Ever_Married_No  Ever_Married_Yes  Graduated_No  ...  \
0          1.0              1.0               0.0           1.0  ...   
1          0.0              0.0               1.0           0.0  ...   
2          0.0              0.0               1.0           0.0  ...   
3          1.0              0.0               1.0           0.0  ...   
4          0.0              0.0               1.0           0.0  ...   

   Spending_Score_Average  Spending_Score_High  Spending_Score_Low  \
0                     

In [15]:
scaler = StandardScaler()
scaled_columns = ['Age', 'Work_Experience', 'Family_Size']
data[scaled_columns] = scaler.fit_transform(data[scaled_columns])

In [16]:
X = data.drop(['Segmentation'], axis=1)
y = data['Segmentation']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
print("\nРозмір тренувальної вибірки:", X_train.shape)
print("Розмір тестової вибірки:", X_test.shape)


Розмір тренувальної вибірки: (6454, 29)
Розмір тестової вибірки: (1614, 29)


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [18]:
#SMOTENC
from imblearn.over_sampling import SMOTENC
from collections import Counter

In [19]:
print(X_train.columns)


Index(['ID', 'Age', 'Work_Experience', 'Family_Size', 'Gender_Female',
       'Gender_Male', 'Ever_Married_No', 'Ever_Married_Yes', 'Graduated_No',
       'Graduated_Yes', 'Profession_Artist', 'Profession_Doctor',
       'Profession_Engineer', 'Profession_Entertainment',
       'Profession_Executive', 'Profession_Healthcare', 'Profession_Homemaker',
       'Profession_Lawyer', 'Profession_Marketing', 'Spending_Score_Average',
       'Spending_Score_High', 'Spending_Score_Low', 'Var_1_Cat_1',
       'Var_1_Cat_2', 'Var_1_Cat_3', 'Var_1_Cat_4', 'Var_1_Cat_5',
       'Var_1_Cat_6', 'Var_1_Cat_7'],
      dtype='object')


In [20]:
categorical_columns = ['Gender_Female', 'Gender_Male', 'Ever_Married_No', 'Ever_Married_Yes',
                       'Graduated_No', 'Graduated_Yes', 'Profession_Artist', 'Profession_Doctor',
                       'Profession_Engineer', 'Profession_Entertainment', 'Profession_Executive',
                       'Profession_Healthcare', 'Profession_Homemaker', 'Profession_Lawyer',
                       'Profession_Marketing', 'Spending_Score_High', 'Spending_Score_Low',
                       'Spending_Score_Average', 'Var_1_Cat_1', 'Var_1_Cat_2', 'Var_1_Cat_3',
                       'Var_1_Cat_4', 'Var_1_Cat_5', 'Var_1_Cat_6', 'Var_1_Cat_7']

categorical_indices = [X_train.columns.get_loc(col) for col in categorical_columns]

In [21]:
print("Баланс класів до SMOTENC:")
print(Counter(y_train))

Баланс класів до SMOTENC:
Counter({'D': 1794, 'C': 1590, 'A': 1581, 'B': 1489})


In [22]:
smotenc = SMOTENC(categorical_features=categorical_indices, random_state=42)

In [23]:
X_train_resampled_smotenc, y_train_resampled_smotenc = smotenc.fit_resample(X_train, y_train)

In [24]:
print("Баланс класів після SMOTENC:")
print(Counter(y_train_resampled_smotenc))

Баланс класів після SMOTENC:
Counter({'A': 1794, 'D': 1794, 'B': 1794, 'C': 1794})


In [25]:
print("Розмір X_train_resampled:", X_train_resampled_smotenc.shape)
print("Розмір y_train_resampled:", y_train_resampled_smotenc.shape)

Розмір X_train_resampled: (7176, 29)
Розмір y_train_resampled: (7176,)


In [26]:
#SMOTE-Tomek

from imblearn.combine import SMOTETomek

In [27]:
print("Баланс класів до SMOTE-Tomek:")
print(Counter(y_train))

Баланс класів до SMOTE-Tomek:
Counter({'D': 1794, 'C': 1590, 'A': 1581, 'B': 1489})


In [28]:
smote_tomek = SMOTETomek(random_state=42)

In [29]:
X_train_resampled_smotetomek, y_train_resampled_smotetomek = smote_tomek.fit_resample(X_train, y_train)

In [30]:
print("Баланс класів після SMOTE-Tomek:")
print(Counter(y_train_resampled_smotetomek))

Баланс класів після SMOTE-Tomek:
Counter({'C': 1382, 'D': 1368, 'B': 1295, 'A': 1283})


In [31]:
print("Розмір X_train_resampled:", X_train_resampled_smotetomek.shape)
print("Розмір y_train_resampled:", y_train_resampled_smotetomek.shape)

Розмір X_train_resampled: (5328, 29)
Розмір y_train_resampled: (5328,)


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier

In [33]:
model = OneVsRestClassifier(LogisticRegression(random_state=42, max_iter=1000))

In [34]:
# Тренування на оригінальних даних
print("Тренування на оригінальних даних...")
model.fit(X_train, y_train)
y_pred_original = model.predict(X_test)
print("\nClassification Report (Original Data):")
print(classification_report(y_test, y_pred_original))

Тренування на оригінальних даних...

Classification Report (Original Data):
              precision    recall  f1-score   support

           A       0.38      0.47      0.42       391
           B       0.38      0.14      0.20       369
           C       0.44      0.60      0.51       380
           D       0.66      0.69      0.67       474

    accuracy                           0.49      1614
   macro avg       0.47      0.47      0.45      1614
weighted avg       0.48      0.49      0.47      1614



In [35]:
# Тренування на даних після SMOTE
print("\nТренування на даних після SMOTE...")
model.fit(X_train_resampled_smotenc, y_train_resampled_smotenc)
y_pred_smote = model.predict(X_test)
print("\nClassification Report (SMOTE Data):")
print(classification_report(y_test, y_pred_smote))


Тренування на даних після SMOTE...

Classification Report (SMOTE Data):
              precision    recall  f1-score   support

           A       0.41      0.49      0.45       391
           B       0.37      0.19      0.25       369
           C       0.45      0.58      0.50       380
           D       0.69      0.67      0.68       474

    accuracy                           0.50      1614
   macro avg       0.48      0.48      0.47      1614
weighted avg       0.49      0.50      0.49      1614



In [36]:
# Тренування на даних після SMOTE-Tomek
print("\nТренування на даних після SMOTE-Tomek...")
model.fit(X_train_resampled_smotetomek, y_train_resampled_smotetomek)
y_pred_smote_tomek = model.predict(X_test)
print("\nClassification Report (SMOTE-Tomek Data):")
print(classification_report(y_test, y_pred_smote_tomek))


Тренування на даних після SMOTE-Tomek...

Classification Report (SMOTE-Tomek Data):
              precision    recall  f1-score   support

           A       0.41      0.46      0.43       391
           B       0.38      0.21      0.27       369
           C       0.44      0.60      0.51       380
           D       0.69      0.67      0.68       474

    accuracy                           0.50      1614
   macro avg       0.48      0.48      0.47      1614
weighted avg       0.49      0.50      0.49      1614



**Висновок**

Мені здається що немає суттєвої різниці між моделями натреновами з SMOTE і SMOTE-Tomek, можливо, тому що не було достатньо репрезентативних точок між класами для того щоб ці методи спрацювали добре.